In [1]:
# list of supported layers
from tensorflow.keras.layers import (
    Input,
    Activation,
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
    Lambda, # only for polynomial activation in the form of `Lambda(lambda x: x**2+x)`
    MaxPooling2D,
    ReLU,
    Softmax,
    )
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# convert y_train and y_test to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
# reshape X_train and X_test to 4D tensor
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [5]:
inputs = Input(shape=(28,28,1))
out = Conv2D(4, 3, use_bias=False)(inputs)
out = BatchNormalization()(out)
out = Lambda(lambda x: x**2+x)(out) # best practice: use polynomial activation instead of ReLU
out = AveragePooling2D()(out) # best practice: use AveragePooling2D instead of MaxPooling2D
out = Conv2D(16, 3, use_bias=False)(out)
out = BatchNormalization()(out)
out = Lambda(lambda x: x**2+x)(out)
out = AveragePooling2D()(out)
out = GlobalAveragePooling2D()(out) # best practice: use GlobalAveragePooling2D instead of Flatten
out = Dense(10, activation=None)(out)
out = Softmax()(out)
model = Model(inputs, out)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 4)         36        
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 4)        16        
 ormalization)                                                   
                                                                 
 lambda (Lambda)             (None, 26, 26, 4)         0         
                                                                 
 average_pooling2d (AverageP  (None, 13, 13, 4)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        576   

In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
    )

In [8]:
model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/100


2023-03-15 21:31:00.844832: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


469/469 [==============================] - 12s 23ms/step - loss: 1.6842 - acc: 0.5150 - val_loss: 1.0291 - val_acc: 0.7401
Epoch 2/100
469/469 [==============================] - 13s 28ms/step - loss: 0.7740 - acc: 0.8326 - val_loss: 0.5402 - val_acc: 0.8735
Epoch 3/100
469/469 [==============================] - 12s 25ms/step - loss: 0.4480 - acc: 0.9004 - val_loss: 0.3657 - val_acc: 0.9106
Epoch 4/100
469/469 [==============================] - 14s 30ms/step - loss: 0.3179 - acc: 0.9244 - val_loss: 0.2855 - val_acc: 0.9289
Epoch 5/100
469/469 [==============================] - 14s 29ms/step - loss: 0.2529 - acc: 0.9365 - val_loss: 0.2198 - val_acc: 0.9443
Epoch 6/100
469/469 [==============================] - 13s 27ms/step - loss: 0.2135 - acc: 0.9436 - val_loss: 0.1831 - val_acc: 0.9532
Epoch 7/100
469/469 [==============================] - 11s 24ms/step - loss: 0.1879 - acc: 0.9488 - val_loss: 0.1799 - val_acc: 0.9508
Epoch 8/100
469/469 [==============================] - 13s 27ms/ste

In [9]:
model.save('best_practice.h5')

In [10]:
model2 = Model(model.input, model.layers[-2].output)
model2.layers[-1]

In [11]:
X = X_test[[0]]
y = model2.predict(X)
y

1/1 [==============================] - 0s 155ms/step


array([[ -6.9723616,  -7.2022758,  -1.5148271,  -5.828606 ,  -8.659952 ,
         -6.311898 , -20.928083 ,  11.584262 , -13.573946 ,  -0.0604342]],
      dtype=float32)

In [12]:
for layer in model.layers:
    print(layer.__class__.__name__, layer.get_config())
    try:
        print(layer.get_config()['function'])
    except:
        pass
    print(layer.get_input_shape_at(0),layer.get_output_shape_at(0))
    try:
        print(layer.get_weights()[0].shape)
        print(layer.get_weights()[1].shape)
    except:
        pass

InputLayer {'batch_input_shape': (None, 28, 28, 1), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_1'}
(None, 28, 28, 1) (None, 28, 28, 1)
Conv2D {'name': 'conv2d', 'trainable': True, 'dtype': 'float32', 'filters': 4, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'linear', 'use_bias': False, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
(None, 28, 28, 1) (None, 26, 26, 4)
(3, 3, 1, 4)
BatchNormalization {'name': 'batch_normalization', 'trainable': True, 'dtype': 'float32', 'axis': ListWrapper([3]), 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer

In [13]:
import json

In [14]:
with open("best_practice.json", "w") as f:
    json.dump({'X': X.flatten().tolist(), 'y': y.flatten().tolist()}, f)